## Perfiles de Aplicaciones Rentables para los Mercados de App Store y Google Play

Nuestro objetivo en este proyecto es encontrar perfiles de aplicaciones móviles que sean rentables para los mercados de App Store y Google Play. Estamos trabajando como analistas de datos para una empresa que construye aplicaciones móviles para Android e iOS, y nuestro trabajo es permitir que nuestro equipo de desarrolladores tome decisiones basadas en datos con respecto al tipo de aplicaciones que construyen.

En nuestra empresa, solo construimos aplicaciones que son gratuitas para descargar e instalar, y nuestra principal fuente de ingresos consiste en anuncios dentro de la aplicación. Esto significa que nuestros ingresos para cualquier aplicación dada están principalmente influenciados por la cantidad de usuarios que utilizan nuestra aplicación. Nuestro objetivo para este proyecto es analizar datos para ayudar a nuestros desarrolladores a comprender qué tipos de aplicaciones son más propensas a atraer a más usuarios.

## Apertura y Exploración de los Datos

Hasta septiembre de 2018, había aproximadamente 2 millones de aplicaciones iOS disponibles en la App Store y 2.1 millones de aplicaciones Android en Google Play.
![img](https://s3.amazonaws.com/dq-content/350/py1m8_statista.png) 

Fuente: [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)

Recopilar datos para más de cuatro millones de aplicaciones requiere una cantidad significativa de tiempo y dinero, así que intentaremos analizar una muestra de datos en su lugar. Para evitar gastar recursos en la recopilación de nuevos datos nosotros mismos, primero deberíamos intentar ver si podemos encontrar algún dato relevante existente sin costo alguno. Afortunadamente, existen dos conjuntos de datos que parecen adecuados para nuestro propósito:

- Un conjunto de datos que contiene información sobre aproximadamente diez mil aplicaciones Android de Google Play. Puedes descargar el conjunto de datos directamente desde este [enlace](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
- Un conjunto de datos que contiene información sobre aproximadamente siete mil aplicaciones iOS de la App Store. Puedes descargar el conjunto de datos directamente desde este [enlace](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Comencemos abriendo los dos conjuntos de datos y luego continuaremos explorando los datos.


In [1]:
from csv import reader

with open("AppleStore.csv") as open_file:
    read_file_apple = reader(open_file)
    list_apple_store = list(read_file_apple)

with open("googleplaystore.csv") as open_file:
    read_file_google = reader(open_file)
    list_google_store = list(read_file_google)

header_google_store = list_google_store[0]
header_apple_store = list_apple_store[0]
android = list_google_store[1:]
ios = list_apple_store[1:]

Para facilitar la exploración de los dos conjuntos de datos, primero escribiremos una función llamada explorar_datos() que podremos usar repetidamente para explorar las filas de una manera más legible. También agregaremos una opción para que nuestra función muestre el número de filas y columnas para cualquier conjunto de datos.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

De los datos anteriormente obtenidos (headers / fila). Podemos determinar que para la "AppleStore" los campos mas importantes son los siguientes: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver' y 'prime_genre'.
Para "GoogleStore": 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price' y 'Genres'.


In [3]:
print(header_apple_store)
explore_data(ios, 0, 1, rows_and_columns=True)
print("\n")
print(header_google_store)
explore_data(android, 0, 1, rows_and_columns=True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7197
Number of columns: 16


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


### Eliminación de Datos Incorrectos

El conjunto de datos de Google Play cuenta con una sección de discusión dedicada, y podemos observar que una de las discusiones destaca un error en la fila 10472. Vamos a imprimir esta fila y compararla con el encabezado y otra fila que esté correcta.


In [4]:
print(len(android))
del (android[10472])
print(len(android))

10841
10840


### Eliminación de Entradas Duplicadas

Parte Uno

Si exploramos el conjunto de datos de Google Play lo suficiente, encontraremos que algunas aplicaciones tienen más de una entrada. Por ejemplo, la aplicación Instagram tiene cuatro entradas:


In [5]:
print("Instagram" in android[0])
for app in android:
    name = app[0]
    if name == "Instagram":
        print(app)

False
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In total, there are 1,181 cases where an app occurs more than once:

In [6]:
unique_apps = []
dupllicate_apps = []

for apps in android:
    name = apps[0]
    if name in unique_apps:
        dupllicate_apps.append(name)
    else:
        unique_apps.append(name)
print(f"El numero de aplicaciones repetidas es: {len(dupllicate_apps)}")



El numero de aplicaciones repetidas es: 1181


Uaremos el siguiente criterio para la eliminacion de datos duplicados: 
- Se dejara vigente el que tenga mayor cantidad de vaoloraciones (corresponde a la fila numero 4 - 'Reviews')

In [7]:
max_reviews = {}
for apps in android:
    name = apps[0]
    n_reviews = float(apps[3])
    if name in max_reviews and max_reviews[name] < n_reviews:
        max_reviews[name] = n_reviews
    elif name not in max_reviews:
        max_reviews[name] = n_reviews

print(f"Cantidad de aplicaciones (incluye duplicados): {len(android)}")
print(f"Cantidad de aplicaciones (sin duplicados): {len(max_reviews)}")
print(f"Se elemino un total de: {len(android) - len(max_reviews)} aplicaciones duplicadas")

Cantidad de aplicaciones (incluye duplicados): 10840
Cantidad de aplicaciones (sin duplicados): 9659
Se elemino un total de: 1181 aplicaciones duplicadas


Para eliminar duplicados según el número máximo de revisiones, seguimos estos pasos:

1. Inicializamos dos listas vacías, `android_clean` y `already_added`.
2. Iteramos a través del conjunto de datos de Android.
3. Para cada aplicación:
    - Aislamos el nombre de la aplicación y el número de revisiones.
    - Añadimos la aplicación actual a la lista `android_clean` y el nombre de la aplicación a la lista `already_added` si:
        - El número de revisiones de la aplicación actual coincide con el número de revisiones de esa aplicación según lo descrito en el diccionario `reviews_max`; y
        - El nombre de la aplicación aún no está en la lista `already_added`.
        Necesitamos agregar esta condición suplementaria para tener en cuenta aquellos casos en los que el número más alto de revisiones de una aplicación duplicada es el mismo para más de una entrada (por ejemplo, la aplicación Box tiene tres entradas y el número de revisiones es el mismo).


In [8]:
android_clean = []
already_added = []
for apps in android:
    name = apps[0]
    n_reviews = float(apps[3])
    if (max_reviews[name] == n_reviews) and (name not in already_added):
        android_clean.append(apps)
        already_added.append(name)



## Eliminación de aplicaciones que no están en inglés
### Parte Uno

Si exploras los conjuntos de datos lo suficiente, notarás que los nombres de algunas de las aplicaciones sugieren que no están dirigidas a un público de habla inglesa. A continuación, vemos un par de ejemplos de ambos conjuntos de datos:


In [23]:
print(ios[813][1])
print(ios[6731][1])

print(android_clean[4412][0])
print(android_clean[794][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
DEM DZ


**No estamos interesados en mantener este tipo de aplicaciones, así que las eliminaremos. Una forma de hacerlo es eliminar cada aplicación cuyo nombre contenga un símbolo que no se utilice comúnmente en el texto en inglés**; el texto en inglés generalmente incluye letras del alfabeto inglés, números compuestos por dígitos del 0 al 9, signos de puntuación (., !, ?, ;, etc.) y otros símbolos (+, *, /, etc.).

**Todos estos caracteres específicos de los textos en inglés se codifican utilizando el estándar ASCII. Cada carácter ASCII tiene un número correspondiente entre 0 y 127 asociado con él, y podemos aprovechar eso para construir una función que verifique un nombre de aplicación y nos diga si contiene caracteres no ASCII.**

**Hemos construido esta función a continuación, y usamos la función incorporada ord() para averiguar el número de codificación correspondiente de cada carácter.**


In [10]:
def is_english(string):
    count = []
    for character in string:
        if ord(character) > 127:
            count.append(character)
    if len(count) > 2:
        return False
    else:
        return True

La función aún no es perfecta y es posible que muy pocas aplicaciones que no están en inglés pasen nuestro filtro, pero parece ser lo suficientemente buena en este punto de nuestro análisis — no deberíamos gastar demasiado tiempo en optimización en este punto.

A continuación, usamos la función is_english() para filtrar las aplicaciones que no están en inglés en ambos conjuntos de datos:


In [11]:
ios_english = []
android_english = []

for apps in android_clean:
    name = apps[0]
    if is_english(name):
        android_english.append(apps)

for apps in ios:
    name = apps[1]
    if is_english(name):
        ios_english.append(apps)

print(ios_english[1])


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


## Aislamiento de las aplicaciones gratuitas

Como mencionamos en la introducción, solo desarrollamos aplicaciones que son gratuitas para descargar e instalar, y nuestra principal fuente de ingresos consiste en anuncios dentro de la aplicación. Nuestros conjuntos de datos contienen tanto aplicaciones gratuitas como de pago, y necesitaremos aislar solo las aplicaciones gratuitas para nuestro análisis. A continuación, aislemos las aplicaciones gratuitas para ambos conjuntos de datos.


In [12]:
android_free = []
ios_free = []

for apps in android_english:
    price = apps[7]
    if price == '0':
        android_free.append(apps)
print(android_free)

for apps in ios_english:
    price = apps[4]
    if price == '0.0':
        ios_free.append(apps)



[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up'], ['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+', '

## Parte Uno

Como mencionamos en la introducción, nuestro objetivo es determinar los tipos de aplicaciones que probablemente atraigan a más usuarios, ya que nuestros ingresos están altamente influenciados por la cantidad de personas que usan nuestras aplicaciones.

Para minimizar riesgos y costos operativos, nuestra estrategia de validación para una idea de aplicación consta de tres pasos:

- Construir una versión mínima de la aplicación para Android y agregarla a Google Play.
- Si la aplicación tiene una buena respuesta de los usuarios, entonces la desarrollamos más.
- Si la aplicación es rentable después de seis meses, también construimos una versión de la aplicación para iOS y la agregamos a la App Store.

Dado que nuestro objetivo final es agregar la aplicación tanto en la App Store como en Google Play, necesitamos encontrar perfiles de aplicaciones que tengan éxito en ambos mercados. Por ejemplo, un perfil que podría funcionar bien para ambos mercados podría ser una aplicación de productividad que haga uso de la gamificación.

Comencemos el análisis obteniendo una idea de los géneros más comunes para cada mercado. Para esto, construiremos una tabla de frecuencias para la columna `prime_genre` del conjunto de datos de la App Store, y las columnas `Genres` y `Category` del conjunto de datos de Google Play.

## Parte Dos

Construiremos dos funciones que podamos usar para analizar las tablas de frecuencias:

1. Una función para generar tablas de frecuencias que muestren porcentajes.
2. Otra función que podamos usar para mostrar los porcentajes en orden descendente.


In [13]:
def freq_table(data_set, index):
    table_freq = {}
    total = 0
    for row in data_set:
        total += 1
        value = row[index]
        if value in table_freq:
            table_freq[value] += 1
        else:
            table_freq[value] = 1
    
    table_percentages = {}
    for key in table_freq:
        percentage = (table_freq[key] / total) * 100
        table_percentages[key] = percentage

    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


**Parte Tres**

Comenzamos examinando la tabla de frecuencias para la columna `prime_genre` del conjunto de datos de la App Store.


In [14]:
display_table(ios_free, -5)

Games : 58.25788323446769
Entertainment : 7.836403371838902
Photo & Video : 4.995316890415236
Education : 3.6840462066812365
Social Networking : 3.3093974399000934
Shopping : 2.5913206369029034
Utilities : 2.466437714642523
Sports : 2.1542304089915705
Music : 2.0605682172962845
Health & Fitness : 2.0293474867311896
Productivity : 1.7483609116453322
Lifestyle : 1.5610365282547611
News : 1.3424914142990947
Travel : 1.248829222603809
Finance : 1.0927255697783327
Weather : 0.8741804558226661
Food & Drink : 0.8117389946924758
Reference : 0.5307524196066188
Business : 0.5307524196066188
Book : 0.3746487667811427
Navigation : 0.18732438339057134
Medical : 0.18732438339057134
Catalogs : 0.1248829222603809


In [15]:
display_table(android_free, 1)

FAMILY : 18.942133815551536
GAME : 9.697106690777577
TOOLS : 8.453887884267631
BUSINESS : 4.599909584086799
PRODUCTIVITY : 3.899186256781193
LIFESTYLE : 3.887884267631103
FINANCE : 3.7070524412296564
MEDICAL : 3.5375226039783
SPORTS : 3.390596745027125
PERSONALIZATION : 3.322784810126582
COMMUNICATION : 3.2323688969258586
HEALTH_AND_FITNESS : 3.0854430379746836
PHOTOGRAPHY : 2.949819168173599
NEWS_AND_MAGAZINES : 2.802893309222423
SOCIAL : 2.667269439421338
TRAVEL_AND_LOCAL : 2.3395117540687163
SHOPPING : 2.2490958408679926
BOOKS_AND_REFERENCE : 2.1360759493670884
DATING : 1.8648282097649187
VIDEO_PLAYERS : 1.7970162748643763
MAPS_AND_NAVIGATION : 1.3901446654611211
FOOD_AND_DRINK : 1.2432188065099457
EDUCATION : 1.164104882459313
ENTERTAINMENT : 0.9606690777576853
LIBRARIES_AND_DEMO : 0.9380650994575045
AUTO_AND_VEHICLES : 0.9267631103074141
HOUSE_AND_HOME : 0.8024412296564195
WEATHER : 0.7911392405063291
EVENTS : 0.7120253164556962
PARENTING : 0.6555153707052441
ART_AND_DESIGN : 0.64

Aún así, las aplicaciones prácticas parecen tener una mejor representación en Google Play en comparación con la App Store. Esta imagen también se confirma con la tabla de frecuencias que vemos para la columna "Genres":

In [16]:
display_table(android_free, -4)

Tools : 8.44258589511754
Entertainment : 6.080470162748644
Education : 5.357142857142857
Business : 4.599909584086799
Productivity : 3.899186256781193
Lifestyle : 3.8765822784810124
Finance : 3.7070524412296564
Medical : 3.5375226039783
Sports : 3.4584086799276674
Personalization : 3.322784810126582
Communication : 3.2323688969258586
Action : 3.096745027124774
Health & Fitness : 3.0854430379746836
Photography : 2.949819168173599
News & Magazines : 2.802893309222423
Social : 2.667269439421338
Travel & Local : 2.328209764918626
Shopping : 2.2490958408679926
Books & Reference : 2.1360759493670884
Simulation : 2.0456600361663653
Dating : 1.8648282097649187
Arcade : 1.842224231464738
Video Players & Editors : 1.7744122965641953
Casual : 1.763110307414105
Maps & Navigation : 1.3901446654611211
Food & Drink : 1.2432188065099457
Puzzle : 1.1301989150090417
Racing : 0.9945750452079566
Role Playing : 0.9380650994575045
Libraries & Demo : 0.9380650994575045
Auto & Vehicles : 0.9267631103074141
St

La diferencia entre las columnas Géneros y Categoría no es del todo clara, pero una cosa que podemos notar es que la columna Géneros es mucho más detallada (tiene más categorías). En este momento, solo estamos buscando tener una idea general, así que solo trabajaremos con la columna Categoría en adelante.

Hasta este punto, encontramos que la App Store está dominada por aplicaciones diseñadas para la diversión, mientras que Google Play muestra un panorama más equilibrado de aplicaciones tanto prácticas como para la diversión. Ahora nos gustaría tener una idea sobre el tipo de aplicaciones que tienen más usuarios.

# Las aplicaciones más populares por género en la App Store

Una forma de averiguar qué géneros son los más populares (tienen más usuarios) es calcular el número promedio de instalaciones para cada género de aplicación. Para el conjunto de datos de Google Play, podemos encontrar esta información en la columna Installs, pero para el conjunto de datos de la App Store esta información está ausente. Como solución alternativa, tomaremos el número total de calificaciones de usuarios como un indicador, que podemos encontrar en la columna `rating_count_tot`.

A continuación, calculamos el número promedio de calificaciones de usuarios por género de aplicación en la App Store:


In [17]:
genres_ios = freq_table(ios_free, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22886.36709539121
Music : 57326.530303030304
Reference : 79350.4705882353
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 19156.493670886077
Travel : 28243.8
Shopping : 27230.734939759037
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16815.48
Entertainment : 14195.358565737051
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 46384.916666666664
Finance : 32367.02857142857
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


En promedio, las aplicaciones de navegación tienen el mayor número de reseñas de usuarios, pero esta cifra está fuertemente influenciada por Waze y Google Maps, que juntas tienen cerca de medio millón de reseñas de usuarios.

In [18]:
for app in ios_free:
    if app[-5] == 'Navigation':
         print(app[1], ':', app[5]) 

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


**Las aplicaciones más populares por género en Google Play**

Para el mercado de Google Play, en realidad tenemos datos sobre el número de instalaciones, por lo que deberíamos poder obtener una imagen más clara sobre la popularidad del género. Sin embargo, los números de instalación no parecen ser lo suficientemente precisos; podemos ver que la mayoría de los valores son abiertos (100+, 1,000+, 5,000+, etc.).


In [19]:
categories_android = freq_table(android_free, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_free:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8814199.78835979
BUSINESS : 1712290.1474201474
COMICS : 832613.8888888889
COMMUNICATION : 38590581.08741259
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1360598.042253521
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1446158.2238372094
GAME : 15544014.51048951
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3650602.276666667
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10830251.970588235
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5145550.285714285
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_MAGAZ

In [20]:
under_100_m = []

for app in android_free:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3617398.420849421

Vemos el mismo patrón para la categoría de reproductores de video, que es el segundo lugar con 24,727,872 instalaciones. El mercado está dominado por aplicaciones como Youtube, Google Play Películas y TV, o MX Player. El patrón se repite para aplicaciones sociales (donde tenemos gigantes como Facebook, Instagram, Google+, etc.), aplicaciones de fotografía (Google Fotos y otros editores de fotos populares), o aplicaciones de productividad (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Nuevamente, la principal preocupación es que estos géneros de aplicaciones podrían parecer más populares de lo que realmente son. Además, estos nichos parecen estar dominados por algunos gigantes difíciles de competir.

El género de juegos parece bastante popular, pero anteriormente descubrimos que esta parte del mercado parece estar un poco saturada, por lo que nos gustaría encontrar una recomendación de aplicación diferente si es posible.

El género de libros y referencia también parece bastante popular, con un número promedio de instalaciones de 8,767,811. Es interesante explorar esto con más profundidad, ya que encontramos que este género tiene cierto potencial para funcionar bien en la App Store, y nuestro objetivo es recomendar un género de aplicación que muestre potencial para ser rentable tanto en la App Store como en Google Play.

Echemos un vistazo a algunas de las aplicaciones de este género y su número de instalaciones:


In [21]:
for app in android_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])


E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

Sin embargo, parece que solo hay algunas aplicaciones muy populares, por lo que este mercado todavía muestra potencial. Intentemos obtener algunas ideas de aplicaciones basadas en el tipo de aplicaciones que se encuentran en algún punto intermedio en cuanto a popularidad (entre 1,000,000 y 100,000,000 descargas):

In [22]:
for app in android_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

Este nicho parece estar dominado por software para el procesamiento y la lectura de libros electrónicos, así como diversas colecciones de bibliotecas y diccionarios, por lo que probablemente no sea una buena idea construir aplicaciones similares debido a la competencia significativa.

También notamos que hay bastantes aplicaciones construidas en torno al libro del Corán, lo que sugiere que construir una aplicación en torno a un libro popular puede ser rentable. Parece que tomar un libro popular (quizás un libro más reciente) y convertirlo en una aplicación podría ser rentable tanto para el mercado de Google Play como para la App Store.

Sin embargo, parece que el mercado ya está lleno de bibliotecas, por lo que necesitamos agregar algunas características especiales además de la versión cruda del libro. Esto podría incluir citas diarias del libro, una versión de audio del libro, cuestionarios sobre el libro, un foro donde las personas puedan discutir el libro, etc.

## Conclusiones

En este proyecto, analizamos datos sobre aplicaciones móviles de la App Store y Google Play con el objetivo de recomendar un perfil de aplicación rentable para ambos mercados.

Concluimos que convertir un libro popular (quizás uno más reciente) en una aplicación podría ser rentable tanto para el mercado de Google Play como para la App Store. Dado que los mercados ya están saturados de aplicaciones de bibliotecas, es necesario agregar características especiales además de la versión básica del libro. Estas características podrían incluir citas diarias del libro, una versión de audio, cuestionarios relacionados, un foro de discusión sobre el libro, entre otras.

